In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit

In [2]:
X_train = pd.read_csv('C:\\Users\\k_abo\\Desktop\\inflation\\X_train_model_2.csv')
y_train = pd.read_csv('C:\\Users\\k_abo\\Desktop\\inflation\\y_train_model_2.csv')
X_train.set_index('Date', inplace = True)
X_train.sort_index(ascending = True, inplace = True)
y_train.set_index('Date', inplace = True)
y_train.sort_index(ascending = True, inplace = True)

In [24]:
xgb0 = xgb.XGBRegressor(
    learning_rate = 0.3,
    n_estimators = 200)

xgbtrain = xgb.DMatrix(X_train, y_train)

cvresults = xgb.cv(
    xgb0.get_params(),
    xgbtrain,
    num_boost_round= xgb0.get_params()['n_estimators'],
    early_stopping_rounds = 50,
    metrics='rmse',
    folds=TimeSeriesSplit(3)
    )
xgb1 = xgb0.set_params(n_estimators = cvresults.shape[0])
xgb1

[18:37:14] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "enable_categorical", "missing", "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:37:14] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "enable_categorical", "missing", "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[18:37:14] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, gamma=None,
             gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.3, max_bin=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=19, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=None,
             reg_alpha=None, reg_lambda=None, ...)

In [25]:
params = {
    'learning_rate':[0.01, 0.1, 0.3, 0.7],
    'max_depth':[1,2,5,7, 8],
    'min_child_weight':range(1,3,1)
}

gridsearch = GridSearchCV(xgb1, param_grid = params, cv = TimeSeriesSplit(3), n_jobs=-1).fit(X_train,y_train)

xgb2 = xgb1.set_params(
    max_depth = gridsearch.best_params_['max_depth'],
    learning_rate = gridsearch.best_params_['learning_rate']
)
best_params = xgb2.fit(X_train, y_train).get_params()
best_params

{'objective': 'reg:squarederror',
 'base_score': 0.5,
 'booster': 'gbtree',
 'callbacks': None,
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'gamma': 0,
 'gpu_id': -1,
 'grow_policy': 'depthwise',
 'importance_type': None,
 'interaction_constraints': '',
 'learning_rate': 0.3,
 'max_bin': 256,
 'max_cat_to_onehot': 4,
 'max_delta_step': 0,
 'max_depth': 1,
 'max_leaves': 0,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 19,
 'n_jobs': 0,
 'num_parallel_tree': 1,
 'predictor': 'auto',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'sampling_method': 'uniform',
 'scale_pos_weight': 1,
 'subsample': 1,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': None}